In [59]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
import uuid
from pandas.io.json import json_normalize
import pycountry
from google.cloud import bigquery
from google.oauth2 import service_account
import odf

In [60]:
#authorization
key_path = "/home/reed/.creds/emissions-team-project.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [61]:
# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level

In [62]:
# **** BIQUERY SCHEMA TABLE SETUP ****

FACTS_TABLE_METADATA = {
    'dim_country': {
        'table_name': 'dim_country',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('country_code', 'string', mode='REQUIRED'),
            bigquery.SchemaField('country_code_2', 'string', mode='NULLABLE'),
            bigquery.SchemaField('country', 'string', mode='NULLABLE'),
            bigquery.SchemaField('land_area_square_km', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('Total_Coastline', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('coastline_per_km_square', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('latitude', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('longitude', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('created_at', 'timestamp', mode='NULLABLE'),
            bigquery.SchemaField('modified_at', 'timestamp', mode='NULLABLE'),
        ]
    }      
}
    


In [63]:
# change to match your filesystem
PROJECT_NAME = "emissions-team-project"
DATASET_NAME = "emissions"

# **** BIGQUERY DATASET CREATION ****

dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

logger.info(f"Created emissions dataset: {dataset.full_dataset_id}")

[DEBUG][2023-01-21 13:14:03,417][retry:0351] : Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
[DEBUG][2023-01-21 13:14:03,442][requests:0192] : Making request: POST https://oauth2.googleapis.com/token
[DEBUG][2023-01-21 13:14:03,444][connectionpool:1003] : Starting new HTTPS connection (1): oauth2.googleapis.com:443
[DEBUG][2023-01-21 13:14:03,569][connectionpool:0456] : https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
[DEBUG][2023-01-21 13:14:03,572][connectionpool:1003] : Starting new HTTPS connection (1): bigquery.googleapis.com:443
[DEBUG][2023-01-21 13:14:03,944][connectionpool:0456] : https://bigquery.googleapis.com:443 "POST /bigquery/v2/projects/emissions-team-project/datasets?prettyPrint=false HTTP/1.1" 409 None
[DEBUG][2023-01-21 13:14:04,258][connectionpool:0456] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/emissions-team-project/datasets/emissions?prettyPrint=false HTTP/1.1" 200 None
[IN

In [65]:
#creating dim_country
df = pd.read_csv('./data/Emissions_by_Country_2002-2022.csv')
lat_long_df = pd.read_csv('./data/world_country_and_usa_states_latitude_and_longitude_values.csv')
df1 = df[['Country', 'ISO 3166-1 alpha-3']]
#make a 2 letter country code
def alpha3code(column):
    CODE=[]
    for country in column:
        try:
            code=pycountry.countries.get(name=country).alpha_3
           # .alpha_3 means 3-letter country code 
           # .alpha_2 means 2-letter country code
            CODE.append(code)
        except:
            CODE.append('None')
    return CODE
    
#make a 3 digit country code column in my lat_long_df and fix the ones that the function missed
lat_long_df['long_code']=alpha3code(lat_long_df.country)
lat_long_df = lat_long_df[['country','long_code','country_code','latitude','longitude']]
lat_long_df = lat_long_df.rename(columns={'country_code': 'country_code_2', 'long_code': 'country_code'})
lat_long_df.at[26,'country_code']='BRN'
lat_long_df.at[27,'country_code']='BOL'
lat_long_df.at[36,'country_code']='CCK'
lat_long_df.at[37,'country_code']='COD'
lat_long_df.at[39,'country_code']='COG'
lat_long_df.at[49,'country_code']='CPV'
lat_long_df.at[52,'country_code']='CZE'
lat_long_df.at[68,'country_code']='FLK'
lat_long_df.at[69,'country_code']='FSM'
lat_long_df.at[105,'country_code']='IRN'
lat_long_df.at[118,'country_code']='PRK'
lat_long_df.at[119,'country_code']='KOR'
lat_long_df.at[123,'country_code']='LAO'
lat_long_df.at[136,'country_code']='MDA'
lat_long_df.at[140,'country_code']='MKD'
lat_long_df.at[142,'country_code']='MMR'
lat_long_df.at[144,'country_code']='MAC'
lat_long_df.at[177,'country_code']='PCN'
lat_long_df.at[179,'country_code']='PSE'
lat_long_df.at[187,'country_code']='RUS'
lat_long_df.at[195,'country_code']='SHN'
lat_long_df.at[204,'country_code']='STP'
lat_long_df.at[206,'country_code']='SYR'
lat_long_df.at[207,'country_code']='SWZ'
lat_long_df.at[222,'country_code']='TWN'
lat_long_df.at[223,'country_code']='TZA'
lat_long_df.at[226,'country_code']='UMI'
lat_long_df.at[230,'country_code']='VAT'
lat_long_df.at[232,'country_code']='VEN'
lat_long_df.at[233,'country_code']='VGB'
lat_long_df.at[234,'country_code']='VIR'
lat_long_df.at[235,'country_code']='VNM'
lat_long_df.at[234,'country_code']='VIR'

#create data frame from coastline_per_country_ods
coastline_df = pd.read_excel("./data/coastline_per_country.ods", engine="odf")
coastline_df = coastline_df[['country_code', 'Total_Coastline','coastline_per_km_square']]

#read country size csv
size_df = pd.read_csv('./data/country_size.csv', dtype=str)
size_df = size_df[['Country Code','2020']]
size_df["2020"] = size_df['2020'].astype('float').round(decimals=0)
size_df = size_df.rename(columns={'2020': 'land_area_square_km', 'Country Code': 'country_code'})

#merge datasets to create dim_countries
dim_country = pd.merge(lat_long_df,size_df, how='left', on='country_code')
dim_country = dim_country[['country', 'country_code','country_code_2','land_area_square_km','latitude','longitude']]
dim_country = pd.merge(dim_country,coastline_df, how='left', on='country_code')
dim_country["created_at"] = pd.Timestamp.now()
dim_country["modified_at"] = None
display(dim_country)


,country,country_code,country_code_2,land_area_square_km,latitude,longitude,Total_Coastline,coastline_per_km_square,created_at,modified_at
0,Andorra,AND,AD,470.0,42.546245,1.601554,0.0,0.00000,2023-01-21 13:14:04.570530,None
1,United Arab Emirates,ARE,AE,71020.0,23.424076,53.847818,1318.0,0.00480,2023-01-21 13:14:04.570530,None
2,Afghanistan,AFG,AF,652230.0,33.939110,67.709953,0.0,0.00000,2023-01-21 13:14:04.570530,None
3,Antigua and Barbuda,ATG,AG,440.0,17.060816,-61.796428,153.0,0.34568,2023-01-21 13:14:04.570530,None
4,Anguilla,AIA,AI,NaN,18.220554,-63.068615,61.0,0.67033,2023-01-21 13:14:04.570530,None
...,...,...,...,...,...,...,...,...,...,...
240,Yemen,YEM,YE,527970.0,15.552727,48.516388,1906.0,0.00417,2023-01-21 13:14:04.570530,None
241,Mayotte,MYT,YT,NaN,-12.827500,45.166244,NaN,NaN,2023-01-21 13:14:04.570530,None
242,South Africa,ZAF,ZA,1213090.0,-30.559482,22.937506,2798.0,0.00482,2023-01-21 13:14:04.570530,None
243,Zambia,ZMB,ZM,743390.0,-13.133897,27.849332,0.0,0.00361,2023-01-21 13:14:04.570530,None


In [66]:
# define the load table function

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [67]:
# Load dim_country dimension table

# get table name and schema from FACTS_TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{FACTS_TABLE_METADATA['dim_country']['table_name']}"
schema = FACTS_TABLE_METADATA['dim_country']['schema']
# load dataframe
load_table(dim_country, client, table_name, schema)

logger.info(f"loaded dim_country facts")

[INFO ][2023-01-21 13:14:04,627][2444475212:0031] : loading table: 'emissions-team-project.emissions.dim_country'
[DEBUG][2023-01-21 13:14:05,788][connectionpool:0456] : https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/emissions-team-project/jobs?uploadType=multipart HTTP/1.1" 200 2444
[DEBUG][2023-01-21 13:14:05,992][connectionpool:0456] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/emissions-team-project/jobs/0f5a583e-df69-4675-a87f-f2388e25654e?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-01-21 13:14:05,994][retry:0214] : Retrying due to , sleeping 0.4s ...
[DEBUG][2023-01-21 13:14:06,609][connectionpool:0456] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/emissions-team-project/jobs/0f5a583e-df69-4675-a87f-f2388e25654e?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-01-21 13:14:06,611][retry:0214] : Retrying due to , sleeping 1.1s ...
[DEBUG][2023-01-21 13:14:07,823][connectionpool:0456] : htt